# Summed Likelihood Analysis with Python

This sample analysis shows a way of performing joint likelihood on two data selections using the same XML model. This is useful if you want to do the following:

* Coanalysis of Front and Back selections (not using the combined IRF)
* Coanalysis of separate time intervals
* Coanalysis of separate energy ranges
* Pass 8 PSF type analysis
* Pass 8 EDISP type analysis

This tutorial also assumes that you've gone through the standard [binned likelihood](https://github.com/fermi-lat/AnalysisThreads/blob/master/SourceAnalysis/1.BinnedLikelihood/binned_likelihood_tutorial.ipynb) thread using the combined front + back events, to which we will compare.

# Get the data

For this thread the original data were extracted from the [LAT data server](https://fermi.gsfc.nasa.gov/cgi-bin/ssc/LAT/LATDataQuery.cgi) with the following selections (these selections are similar to those in the paper):

```
Search Center (RA,Dec) = (193.98,-5.82)
Radius = 15 degrees
Start Time (MET) = 239557417 seconds (2008-08-04T15:43:37)
Stop Time (MET) = 302572802 seconds (2010-08-04T00:00:00)
Minimum Energy = 100 MeV
Maximum Energy = 500000 MeV
```


These are the event files. Run the code cell below to retrieve them:
```
L181126210218F4F0ED2738_PH00.fits (5.4 MB)
L181126210218F4F0ED2738_PH01.fits (10.8 MB)
L181126210218F4F0ED2738_PH02.fits (6.9 MB)
L181126210218F4F0ED2738_PH03.fits (9.8 MB)
L181126210218F4F0ED2738_PH04.fits (7.8 MB)
L181126210218F4F0ED2738_PH05.fits (6.6 MB)
L181126210218F4F0ED2738_PH06.fits (4.8 MB)
L181126210218F4F0ED2738_SC00.fits (256 MB spacecraft file)
```

In [1]:
!wget https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/BinnedLikelihood/L181126210218F4F0ED2738_PH00.fits
!wget https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/BinnedLikelihood/L181126210218F4F0ED2738_PH01.fits
!wget https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/BinnedLikelihood/L181126210218F4F0ED2738_PH02.fits
!wget https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/BinnedLikelihood/L181126210218F4F0ED2738_PH03.fits
!wget https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/BinnedLikelihood/L181126210218F4F0ED2738_PH04.fits
!wget https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/BinnedLikelihood/L181126210218F4F0ED2738_PH05.fits
!wget https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/BinnedLikelihood/L181126210218F4F0ED2738_PH06.fits
!wget https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/BinnedLikelihood/L181126210218F4F0ED2738_SC00.fits

--2025-09-03 12:16:41--  https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/BinnedLikelihood/L181126210218F4F0ED2738_PH00.fits
Resolving fermi.gsfc.nasa.gov (fermi.gsfc.nasa.gov)... 129.164.179.26
Connecting to fermi.gsfc.nasa.gov (fermi.gsfc.nasa.gov)|129.164.179.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5189760 (4.9M) [application/fits]
Saving to: ‘L181126210218F4F0ED2738_PH00.fits’

L181126210218F4F0ED 100%[===================>]   4.95M  2.17MB/s    in 2.3s    

2025-09-03 12:16:44 (2.17 MB/s) - ‘L181126210218F4F0ED2738_PH00.fits’ saved [5189760/5189760]

--2025-09-03 12:16:44--  https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/BinnedLikelihood/L181126210218F4F0ED2738_PH01.fits
Resolving fermi.gsfc.nasa.gov (fermi.gsfc.nasa.gov)... 129.164.179.26
Connecting to fermi.gsfc.nasa.gov (fermi.gsfc.nasa.gov)|129.164.179.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10995840 (10M) [application/fits]
Saving

In [2]:
!mkdir data
!mv *.fits ./data

mkdir: data: File exists


You'll first need to make a file list with the names of your input event files:

In [3]:
!ls ./data/*_PH*.fits > ./data/binned_events.txt
!cat ./data/binned_events.txt

./data/L181126210218F4F0ED2738_PH00.fits
./data/L181126210218F4F0ED2738_PH01.fits
./data/L181126210218F4F0ED2738_PH02.fits
./data/L181126210218F4F0ED2738_PH03.fits
./data/L181126210218F4F0ED2738_PH04.fits
./data/L181126210218F4F0ED2738_PH05.fits
./data/L181126210218F4F0ED2738_PH06.fits


In the following analysis we've assumed that you've named your list of data files `binned_events.txt`.

# Perform Event Selections

You could follow the unbinned likelihood tutorial to perform your event selections using **gtlike*, **gtmktime**, etc. directly from the command line, and then use pylikelihood later.

But we're going to go ahead and use python. The `gt_apps` module provides methods to call these tools from within python. This'll get us used to using python.

So, let's jump into python:

In [4]:
import gt_apps as my_apps

We need to run **gtselect** (called `filter` in python) twice. Once, we select only the front events and the other time we select only back events. You do this with `evtype=1` (front) and `evtype=2` (back).

In [5]:
my_apps.filter['evclass'] = 128
my_apps.filter['evtype'] = 1
my_apps.filter['ra'] = 193.98
my_apps.filter['dec'] = -5.82
my_apps.filter['rad'] = 15
my_apps.filter['emin'] = 100
my_apps.filter['emax'] = 500000
my_apps.filter['zmax'] = 90
my_apps.filter['tmin'] = 239557417
my_apps.filter['tmax'] = 302572802
my_apps.filter['infile'] = '@./data/binned_events.txt'
my_apps.filter['outfile'] = './data/3C279_front_filtered.fits'

Once this is done, we can run **gtselect**:

In [6]:
my_apps.filter.run()

time -p gtselect infile=@./data/binned_events.txt outfile=./data/3C279_front_filtered.fits ra=193.98 dec=-5.82 rad=15.0 tmin=239557417.0 tmax=302572802.0 emin=100.0 emax=500000.0 zmin=0.0 zmax=90.0 evclass=128 evtype=1 convtype=-1 phasemin=0.0 phasemax=1.0 evtable="EVENTS" chatter=2 clobber=yes debug=no gui=no mode="ql"
Done.
real 2.68
user 2.13
sys 0.15


Now, we select the back events and run it again:

In [7]:
my_apps.filter['evtype'] = 2
my_apps.filter['outfile'] = './data/3C279_back_filtered.fits'

In [8]:
my_apps.filter.run()

time -p gtselect infile=@./data/binned_events.txt outfile=./data/3C279_back_filtered.fits ra=193.98 dec=-5.82 rad=15.0 tmin=239557417.0 tmax=302572802.0 emin=100.0 emax=500000.0 zmin=0.0 zmax=90.0 evclass=128 evtype=2 convtype=-1 phasemin=0.0 phasemax=1.0 evtable="EVENTS" chatter=2 clobber=yes debug=no gui=no mode="ql"
Done.
real 1.91
user 1.78
sys 0.09


Now, we need to find the GTIs for each data set (front and back). This is accessed within python via the `maketime` object:

In [9]:
# Front
my_apps.maketime['scfile'] = './data/L181126210218F4F0ED2738_SC00.fits'
my_apps.maketime['filter'] = '(DATA_QUAL>0)&&(LAT_CONFIG==1)'
my_apps.maketime['roicut'] = 'no'
my_apps.maketime['evfile'] = './data/3C279_front_filtered.fits'
my_apps.maketime['outfile'] = './data/3C279_front_filtered_gti.fits'

In [10]:
my_apps.maketime.run()

time -p gtmktime scfile=./data/L181126210218F4F0ED2738_SC00.fits sctable="SC_DATA" filter="(DATA_QUAL>0)&&(LAT_CONFIG==1)" roicut=no evfile=./data/3C279_front_filtered.fits evtable="EVENTS" outfile="./data/3C279_front_filtered_gti.fits" apply_filter=yes overwrite=no header_obstimes=yes tstart=0.0 tstop=0.0 gtifile="default" chatter=2 clobber=yes debug=no gui=no mode="ql"
real 12.64
user 12.08
sys 0.34


Similar for the back:

In [11]:
# Back
my_apps.maketime['evfile'] = './data/3C279_back_filtered.fits'
my_apps.maketime['outfile'] = './data/3C279_back_filtered_gti.fits'

In [12]:
my_apps.maketime.run()

time -p gtmktime scfile=./data/L181126210218F4F0ED2738_SC00.fits sctable="SC_DATA" filter="(DATA_QUAL>0)&&(LAT_CONFIG==1)" roicut=no evfile=./data/3C279_back_filtered.fits evtable="EVENTS" outfile="./data/3C279_back_filtered_gti.fits" apply_filter=yes overwrite=no header_obstimes=yes tstart=0.0 tstop=0.0 gtifile="default" chatter=2 clobber=yes debug=no gui=no mode="ql"
real 12.81
user 12.27
sys 0.29


# Livetime and Counts Cubes

### Livetime Cube

We can now compute the livetime cube. We only need to do this once since in this case we made the exact same time cuts and used the same GTI filter on front and back datasets.

In [13]:
my_apps.expCube['evfile'] = './data/3C279_front_filtered_gti.fits'
my_apps.expCube['scfile'] = './data/L181126210218F4F0ED2738_SC00.fits'
my_apps.expCube['outfile'] = './data/3C279_front_ltcube.fits'
my_apps.expCube['zmax'] = 90
my_apps.expCube['dcostheta'] = 0.025
my_apps.expCube['binsz'] = 1

In [14]:
my_apps.expCube.run()

time -p gtltcube evfile="./data/3C279_front_filtered_gti.fits" evtable="EVENTS" scfile=./data/L181126210218F4F0ED2738_SC00.fits sctable="SC_DATA" outfile=./data/3C279_front_ltcube.fits dcostheta=0.025 binsz=1.0 phibins=0 tmin=0.0 tmax=0.0 file_version="1" zmin=0.0 zmax=90.0 chatter=2 clobber=yes debug=no gui=no mode="ql"
Working on file ./data/L181126210218F4F0ED2738_SC00.fits
.....................!
real 1117.24
user 1099.98
sys 4.75


### Counts Cube

The counts cube is the counts from our data file binned in space and energy. All of the steps above use a circular ROI (or a cone, really).

Once you switch to binned analysis, you start doing things in squares. Your counts cube can only be as big as the biggest square that can fit in the circular ROI you already selected.

We start with front events:

In [15]:
my_apps.evtbin['evfile'] = './data/3C279_front_filtered_gti.fits'
my_apps.evtbin['outfile'] = './data/3C279_front_ccube.fits'
my_apps.evtbin['algorithm'] = 'CCUBE'
my_apps.evtbin['nxpix'] = 100
my_apps.evtbin['nypix'] = 100
my_apps.evtbin['binsz'] = 0.2
my_apps.evtbin['coordsys'] = 'CEL'
my_apps.evtbin['xref'] = 193.98
my_apps.evtbin['yref'] = -5.82
my_apps.evtbin['axisrot'] = 0
my_apps.evtbin['proj'] = 'AIT'
my_apps.evtbin['ebinalg'] = 'LOG'
my_apps.evtbin['emin'] = 100
my_apps.evtbin['emax'] = 500000
my_apps.evtbin['enumbins'] = 37

In [16]:
my_apps.evtbin.run()

time -p gtbin evfile=./data/3C279_front_filtered_gti.fits scfile=NONE outfile=./data/3C279_front_ccube.fits algorithm="CCUBE" ebinalg="LOG" emin=100.0 emax=500000.0 enumbins=37 denergy=0.0 ebinfile=__energyBins.fits tbinalg="LIN" tstart=243215766.614 tstop=243217766.614 dtime=4.096 tbinfile=NONE snratio=0.0 lcemin=0.0 lcemax=0.0 nxpix=100 nypix=100 binsz=0.2 coordsys="CEL" xref=193.98 yref=-5.82 axisrot=0.0 rafield="RA" decfield="DEC" proj="AIT" hpx_ordering_scheme="RING" hpx_order=3 hpx_ebin=yes hpx_region="" evtable="EVENTS" sctable="SC_DATA" efield="ENERGY" tfield="TIME" chatter=2 clobber=yes debug=no gui=no mode="ql"
This is gtbin version HEAD
gtbin: WARNING: No spacecraft file: EXPOSURE keyword will be set equal to ontime.
real 0.56
user 0.27
sys 0.08


And then for the back events:

In [17]:
my_apps.evtbin['evfile'] = './data/3C279_back_filtered_gti.fits'
my_apps.evtbin['outfile'] = './data/3C279_back_ccube.fits'

In [18]:
my_apps.evtbin.run()

time -p gtbin evfile=./data/3C279_back_filtered_gti.fits scfile=NONE outfile=./data/3C279_back_ccube.fits algorithm="CCUBE" ebinalg="LOG" emin=100.0 emax=500000.0 enumbins=37 denergy=0.0 ebinfile=__energyBins.fits tbinalg="LIN" tstart=243215766.614 tstop=243217766.614 dtime=4.096 tbinfile=NONE snratio=0.0 lcemin=0.0 lcemax=0.0 nxpix=100 nypix=100 binsz=0.2 coordsys="CEL" xref=193.98 yref=-5.82 axisrot=0.0 rafield="RA" decfield="DEC" proj="AIT" hpx_ordering_scheme="RING" hpx_order=3 hpx_ebin=yes hpx_region="" evtable="EVENTS" sctable="SC_DATA" efield="ENERGY" tfield="TIME" chatter=2 clobber=yes debug=no gui=no mode="ql"
This is gtbin version HEAD
gtbin: WARNING: No spacecraft file: EXPOSURE keyword will be set equal to ontime.
real 0.31
user 0.25
sys 0.03


# Exposure Maps

The binned exposure map is an exposure map binned in space and energy.

We first need to import the python version of `gtexpcube2`, which doesn't have a gtapp version by default. This is easy to do (you can import any of the command line tools into python this way). Then, you can check out the parameters with the `pars` function.

In [19]:
from GtApp import GtApp
expCube2= GtApp('gtexpcube2','Likelihood')
expCube2.pars()

' infile=./data/3C279_front_ltcube.fits cmap=none outfile=./data/3C279_back_BinnedExpMap.fits irfs="P8R3_SOURCE_V2" evtype="INDEF" edisp_bins=0 nxpix=1800 nypix=900 binsz=0.2 coordsys="CEL" xref=193.98 yref=-5.82 axisrot=0.0 proj="AIT" ebinalg="LOG" emin=100.0 emax=500000.0 enumbins=37 ebinfile="NONE" hpx_ordering_scheme="RING" hpx_order=6 bincalc="EDGE" ignorephi=no thmax=180.0 thmin=0.0 table="EXPOSURE" chatter=2 clobber=yes debug=no mode="ql"'

Here, we generate exposure maps for the entire sky. 

In [20]:
expCube2['infile'] = './data/3C279_front_ltcube.fits'
expCube2['cmap'] = 'none'
expCube2['outfile'] = './data/3C279_front_BinnedExpMap.fits'
expCube2['irfs'] = 'P8R3_SOURCE_V2'
expCube2['evtype'] = '1'
expCube2['nxpix'] = 1800
expCube2['nypix'] = 900
expCube2['binsz'] = 0.2
expCube2['coordsys'] = 'CEL'
expCube2['xref'] = 193.98
expCube2['yref'] = -5.82
expCube2['axisrot'] = 0
expCube2['proj'] = 'AIT'
expCube2['ebinalg'] = 'LOG'
expCube2['emin'] = 100
expCube2['emax'] = 500000
expCube2['enumbins'] = 37

In [21]:
expCube2.run()

time -p gtexpcube2 infile=./data/3C279_front_ltcube.fits cmap=none outfile=./data/3C279_front_BinnedExpMap.fits irfs="P8R3_SOURCE_V2" evtype=1 edisp_bins=0 nxpix=1800 nypix=900 binsz=0.2 coordsys="CEL" xref=193.98 yref=-5.82 axisrot=0.0 proj="AIT" ebinalg="LOG" emin=100.0 emax=500000.0 enumbins=37 ebinfile="NONE" hpx_ordering_scheme="RING" hpx_order=6 bincalc="EDGE" ignorephi=no thmax=180.0 thmin=0.0 table="EXPOSURE" chatter=2 clobber=yes debug=no mode="ql"

******************************************
    Newer IRF version available. P8R3_SOURCE_V3
******************************************

Computing binned exposure map....................!
real 54.76
user 53.51
sys 0.98


In [22]:
expCube2['infile'] = './data/3C279_front_ltcube.fits'
expCube2['outfile'] = './data/3C279_back_BinnedExpMap.fits'
expCube2['evtype'] = '2'

In [23]:
expCube2.run()

time -p gtexpcube2 infile=./data/3C279_front_ltcube.fits cmap=none outfile=./data/3C279_back_BinnedExpMap.fits irfs="P8R3_SOURCE_V2" evtype=2 edisp_bins=0 nxpix=1800 nypix=900 binsz=0.2 coordsys="CEL" xref=193.98 yref=-5.82 axisrot=0.0 proj="AIT" ebinalg="LOG" emin=100.0 emax=500000.0 enumbins=37 ebinfile="NONE" hpx_ordering_scheme="RING" hpx_order=6 bincalc="EDGE" ignorephi=no thmax=180.0 thmin=0.0 table="EXPOSURE" chatter=2 clobber=yes debug=no mode="ql"

******************************************
    Newer IRF version available. P8R3_SOURCE_V3
******************************************

Computing binned exposure map....................!
real 53.41
user 52.33
sys 0.94


# Compute Source Maps

The source maps step convolves the LAT response with your source model, generating maps for each source in the model for use in the likelihood calculation.

We use the same [XML](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/BinnedLikelihood/3C279_input_model.xml) file as in the standard [binned likelihood](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/binned_likelihood_tutorial.html) analysis.

If you want to make your own model filey, you should also download the recommended models for a normal point source analysis `gll_iem_v07.fits` and `iso_P8R3_SOURCE_V3_v1.txt`.

We'll take a shortcut, however, and simply copy the `3C279_input_model.xml` we made in the Binned Likelihood notebook.

In [24]:
!wget https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/BinnedLikelihood/3C279_input_model.xml

--2025-09-03 12:40:09--  https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/BinnedLikelihood/3C279_input_model.xml
Resolving fermi.gsfc.nasa.gov (fermi.gsfc.nasa.gov)... 129.164.179.26
Connecting to fermi.gsfc.nasa.gov (fermi.gsfc.nasa.gov)|129.164.179.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 184526 (180K) [application/xml]
Saving to: ‘3C279_input_model.xml’

3C279_input_model.x 100%[===================>] 180.20K   629KB/s    in 0.3s    

2025-09-03 12:40:10 (629 KB/s) - ‘3C279_input_model.xml’ saved [184526/184526]



In [25]:
!mv *.xml ./data

Note that the files `gll_iem_v07.fits` and `iso_P8R3_SOURCE_V2_v1.txt` must be in your current working directory for the next steps to work.

We compute the front events:

In [26]:
my_apps.srcMaps['expcube'] = './data/3C279_front_ltcube.fits'
my_apps.srcMaps['cmap'] = './data/3C279_front_ccube.fits'
my_apps.srcMaps['srcmdl'] = './data/3C279_input_model.xml'
my_apps.srcMaps['bexpmap'] = './data/3C279_front_BinnedExpMap.fits'
my_apps.srcMaps['outfile'] = './data/3C279_front_srcmap.fits'
my_apps.srcMaps['irfs'] = 'P8R3_SOURCE_V3'
my_apps.srcMaps['evtype'] = '1'

In [27]:
my_apps.srcMaps.run()

time -p gtsrcmaps scfile= sctable="SC_DATA" expcube=./data/3C279_front_ltcube.fits cmap=./data/3C279_front_ccube.fits srcmdl=./data/3C279_input_model.xml bexpmap=./data/3C279_front_BinnedExpMap.fits wmap=none outfile=./data/3C279_front_srcmap.fits irfs="P8R3_SOURCE_V3" evtype=1 convol=yes resample=yes rfactor=2 minbinsz=0.1 ptsrc=yes psfcorr=yes emapbnds=yes edisp_bins=0 copyall=no chatter=2 clobber=yes debug=no gui=no mode="ql"
Generating SourceMap for 4FGL J1118.2-0415 38....................!
Generating SourceMap for 4FGL J1118.6-1235 38....................!
Generating SourceMap for 4FGL J1119.9-1007 38....................!
Generating SourceMap for 4FGL J1121.3-0011 38....................!
Generating SourceMap for 4FGL J1121.4-0553 38....................!
Generating SourceMap for 4FGL J1122.0-0231 38....................!
Generating SourceMap for 4FGL J1122.5-1440 38....................!
Generating SourceMap for 4FGL J1124.1-1203 38....................!
Generating SourceMap for 4FGL J

And similarly, the back events:

In [28]:
my_apps.srcMaps['expcube'] = './data/3C279_front_ltcube.fits'
my_apps.srcMaps['cmap'] = './data/3C279_back_ccube.fits'
my_apps.srcMaps['srcmdl'] = './data/3C279_input_model.xml'
my_apps.srcMaps['bexpmap'] = './data/3C279_back_BinnedExpMap.fits'
my_apps.srcMaps['outfile'] = './data/3C279_back_srcmap.fits'
my_apps.srcMaps['irfs'] = 'P8R3_SOURCE_V3'
my_apps.srcMaps['evtype'] = '2'

In [29]:
my_apps.srcMaps.run()

time -p gtsrcmaps scfile= sctable="SC_DATA" expcube=./data/3C279_front_ltcube.fits cmap=./data/3C279_back_ccube.fits srcmdl=./data/3C279_input_model.xml bexpmap=./data/3C279_back_BinnedExpMap.fits wmap=none outfile=./data/3C279_back_srcmap.fits irfs="P8R3_SOURCE_V3" evtype=2 convol=yes resample=yes rfactor=2 minbinsz=0.1 ptsrc=yes psfcorr=yes emapbnds=yes edisp_bins=0 copyall=no chatter=2 clobber=yes debug=no gui=no mode="ql"
Generating SourceMap for 4FGL J1118.2-0415 38....................!
Generating SourceMap for 4FGL J1118.6-1235 38....................!
Generating SourceMap for 4FGL J1119.9-1007 38....................!
Generating SourceMap for 4FGL J1121.3-0011 38....................!
Generating SourceMap for 4FGL J1121.4-0553 38....................!
Generating SourceMap for 4FGL J1122.0-0231 38....................!
Generating SourceMap for 4FGL J1122.5-1440 38....................!
Generating SourceMap for 4FGL J1124.1-1203 38....................!
Generating SourceMap for 4FGL J112

# Run the Likelihood Analysis

First, import the BinnedAnalysis and SummedAnalysis libraries. Then, create a likelihood object for both the front and the back datasets. For more details on the pyLikelihood module, check out the [pyLikelihood Usage Notes](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/python_usage_notes.html).

In [30]:
import pyLikelihood
from BinnedAnalysis import *
from SummedLikelihood import *

front = BinnedObs(srcMaps='./data/3C279_front_srcmap.fits',binnedExpMap='./data/3C279_front_BinnedExpMap.fits',expCube='./data/3C279_front_ltcube.fits',irfs='CALDB')
likefront = BinnedAnalysis(front,'./data/3C279_input_model.xml',optimizer='NewMinuit')
back = BinnedObs(srcMaps='./data/3C279_back_srcmap.fits',binnedExpMap='./data/3C279_back_BinnedExpMap.fits',expCube='./data/3C279_front_ltcube.fits',irfs='CALDB')
likeback = BinnedAnalysis(back,'./data/3C279_input_model.xml',optimizer='NewMinuit')

Then, create the summedlikelihood object and add the two likelihood objects, one for the front selection and the second for the back selection.

In [31]:
summed_like = SummedLikelihood()
summed_like.addComponent(likefront)
summed_like.addComponent(likeback)

Perform the fit and print out the results:

In [32]:
summedobj = pyLike.NewMinuit(summed_like.logLike)
summed_like.fit(verbosity=0,covar=True,optObject=summedobj)

214779.04004262824

Print TS for 3C 279 (4FGL J1256.1-0547):

In [33]:
summed_like.Ts('4FGL J1256.1-0547')

29379.24933748087

We can now compare to the standard [binned likelihood](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/binned_likelihood_tutorial.html) analysis that uses only one data set containing both Front and Back event types that are represented by a single, combined IRF set. You will need to download the files created in that analysis thread or rerun this python tutorial with the combined dataset `(evtype=3)`.

For your convenience, the files can be obtained from the code cell below:

In [34]:
!wget https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/BinnedLikelihood/3C279_binned_srcmaps.fits
!wget https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/BinnedLikelihood/3C279_binned_allsky_expcube.fits
!wget https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/BinnedLikelihood/3C279_binned_ltcube.fits

--2025-09-03 12:49:12--  https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/BinnedLikelihood/3C279_binned_srcmaps.fits
Resolving fermi.gsfc.nasa.gov (fermi.gsfc.nasa.gov)... 129.164.179.26
Connecting to fermi.gsfc.nasa.gov (fermi.gsfc.nasa.gov)|129.164.179.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1681920 (1.6M) [application/fits]
Saving to: ‘3C279_binned_srcmaps.fits’

3C279_binned_srcmap 100%[===================>]   1.60M   789KB/s    in 2.1s    

2025-09-03 12:49:15 (789 KB/s) - ‘3C279_binned_srcmaps.fits’ saved [1681920/1681920]

--2025-09-03 12:49:15--  https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/BinnedLikelihood/3C279_binned_allsky_expcube.fits
Resolving fermi.gsfc.nasa.gov (fermi.gsfc.nasa.gov)... 129.164.179.26
Connecting to fermi.gsfc.nasa.gov (fermi.gsfc.nasa.gov)|129.164.179.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 246438720 (235M) [application/fits]
Saving to: ‘3C279_binned_allsk

In [35]:
!mv 3C279*.fits ./data

In [36]:
all = BinnedObs(srcMaps='./data/3C279_binned_srcmaps.fits',binnedExpMap='./data/3C279_binned_allsky_expcube.fits',expCube='./data/3C279_binned_ltcube.fits',irfs='CALDB')
likeall = BinnedAnalysis(all,'./data/3C279_input_model.xml',optimizer='NewMinuit')


******************************************
    Newer IRF version available. P8R3_SOURCE_V3
******************************************

Generating SourceMap for 4FGL J1118.2-0415 38....................!
Generating SourceMap for 4FGL J1118.6-1235 38....................!
Generating SourceMap for 4FGL J1119.9-1007 38....................!
Generating SourceMap for 4FGL J1121.3-0011 38....................!
Generating SourceMap for 4FGL J1121.4-0553 38....................!
Generating SourceMap for 4FGL J1122.0-0231 38....................!
Generating SourceMap for 4FGL J1122.5-1440 38....................!
Generating SourceMap for 4FGL J1124.1-1203 38....................!
Generating SourceMap for 4FGL J1124.5-0658 38....................!
Generating SourceMap for 4FGL J1124.6-0809 38....................!
Generating SourceMap for 4FGL J1125.9-0742 38....................!
Generating SourceMap for 4FGL J1129.2-0529 38....................!
Generating SourceMap for 4FGL J1129.2-1014 38...............

Perform the fit and print out the results:

In [37]:
likeallobj = pyLike.NewMinuit(likeall.logLike)
likeall.fit(verbosity=0,covar=True,optObject=likeallobj)

Generating SourceMap for 4FGL J1229.0+0202 38....................!
Generating SourceMap for 4FGL J1243.9-0218 38....................!
Generating SourceMap for 4FGL J1249.3-0545 38....................!
Generating SourceMap for 4FGL J1251.3-0201 38....................!
Generating SourceMap for 4FGL J1256.1-0547 38....................!
Generating SourceMap for 4FGL J1258.7-0452 38....................!
Generating SourceMap for 4FGL J1304.6-0348 38....................!
Generating SourceMap for 4FGL J1310.1-0450 38....................!
Generating SourceMap for 4FGL J1312.8-0425 38....................!
Generating SourceMap for gll_iem_v07 38....................!
Generating SourceMap for iso_P8R3_SOURCE_V3_v1 38....................!


73159.31353785182

Print TS for 3C 279 (4FGL J1256.1-0547):

In [38]:
likeall.Ts('4FGL J1256.1-0547')

28631.56329077939

The TS for the front + back analysis is slightly lower than what we found for the separate front and back analysis.

The important difference is that in the separated version of the analysis each event type has a dedicated response function set instead of using the averaged Front+Back response. This should increase the sensitivity, and therefore, the TS value.